# Notebook to test the Bedrock Agents and Unity Catalog Functions.

# Register Toolkit Functions

In [1]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration
from unitycatalog.ai.bedrock.toolkit import UCFunctionToolkit

In [2]:
UNITY_CATALOG_SERVER = "localhost:8080"
CATALOG = "AICatalog"
SCHEMA = "AISchema"

In [3]:
config = Configuration()
config.host = f"http://{UNITY_CATALOG_SERVER}/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

uc_client = UnitycatalogFunctionClient(api_client=api_client)


In [4]:
import random
from datetime import datetime

def get_weather_in_celsius(location_id: str, 
                           fetch_date: str
                           ) -> str:
    """
    Simulate fetching weather data (in Celsius) for a given location and date.

    Args:
        location_id (str): A unique identifier for the location.
        fetch_date (str): The date to fetch the weather for (format: YYYY-MM-DD).

    Raises:
        ValueError: If inputs are invalid (e.g., invalid date format).
        Exception: For unexpected errors during execution.

    Returns:
        float: Simulated temperature in Celsius.
    """
    try:
        # Validate location_id
        if not isinstance(location_id, str) or not location_id.strip():
            raise ValueError("location_id must be a non-empty string.")
        
        # Validate and parse fetch_date
        try:
            datetime.strptime(fetch_date, "%Y-%m-%d")
        except ValueError:
            raise ValueError("fetch_date must be in the format YYYY-MM-DD.")
        
        min_f: float = -30.0, 
        max_f: float = 120.0
        # Generate a random temperature in Fahrenheit
        fahrenheit = round(random.uniform(min_f, max_f), 2)

        # Convert to Celsius
        celsius = round((fahrenheit - 32) * 5 / 9, 2)

        return str(celsius)

    except ValueError as ve:
        raise ve  # Re-raise the specific ValueError for invalid inputs
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}")


In [29]:
uc_client.uc.create_catalog(name=CATALOG, comment="Catalog for AI functions")

CatalogInfo(name='AICatalog', comment='Catalog for AI functions', properties={}, owner=None, created_at=1737005234233, created_by=None, updated_at=1737005234233, updated_by=None, id='7dc1e000-2854-46ac-9811-d140c1cb942e')

In [30]:
uc_client.uc.create_schema(catalog_name=CATALOG, name=SCHEMA, comment="Schema for AI functions")

SchemaInfo(name='AISchema', catalog_name='AICatalog', comment='Schema for AI functions', properties={}, full_name='AICatalog.AISchema', owner=None, created_at=1737005278533, created_by=None, updated_at=1737005278533, updated_by=None, schema_id='c164dc56-8ee1-489c-9dad-19734bbcb0d7')

In [5]:
uc_client.create_python_function(
    func=get_weather_in_celsius, 
    catalog=CATALOG, 
    schema=SCHEMA, 
    replace=True
)

/Users/sreeramreddy.thoom/oss-work/forked_uc_ai/.venv/lib/python3.13/site-packages/unitycatalog/ai/core/utils/callable_utils.py:421: UserWarning: In function 'get_weather_in_celsius': The following parameters are documented in the docstring but not present in the function signature: Exception, ValueError
  check_docstring_signature_consistency(docstring_info.params, params_in_signature, func_name)


FunctionInfo(name='get_weather_in_celsius', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "A unique identifier for the location."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='A unique identifier for the location.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date to fetch the weather for (format: YYYY-MM-DD)."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The dat

In [6]:
uc_client.list_functions(catalog=CATALOG, schema=SCHEMA)

[FunctionInfo(name='bedrock_test_function', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='name', type_text='STRING', type_json='{"name": "name", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL', routine_definition='try:\n    # Fetch from Databricks SQL Warehouse based UC function execution \n    return "hello: " + name\nexcept Exception as e:\n    raise Exception(f"Error occurred: {e}")', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', i

In [7]:
function_name = f"{CATALOG}.{SCHEMA}.get_weather_in_celsius"
uc_f_toolkit = UCFunctionToolkit(function_names=[function_name], client=uc_client)

In [11]:

print(uc_f_toolkit)

UCFunctionToolkit(function_names=['AICatalog.AISchema.get_weather_in_celsius'], tools_dict={'AICatalog.AISchema.get_weather_in_celsius': BedrockTool(name='AICatalog__AISchema__get_weather_in_celsius', description='Simulate fetching weather data (in Celsius) for a given location and date.', parameters={'type': 'object', 'properties': {'location_id': {'default': None, 'description': 'A unique identifier for the location.', 'title': 'Location Id', 'type': 'string'}, 'fetch_date': {'default': None, 'description': 'The date to fetch the weather for (format: YYYY-MM-DD).', 'title': 'Fetch Date', 'type': 'string'}}, 'required': []}, requireConfirmation='ENABLED')}, client=<unitycatalog.ai.core.client.UnitycatalogFunctionClient object at 0x117088ec0>)


# Create AWS Bedrock Agent and Agent Action Group

In [12]:
import boto3, pprint, json, time, uuid

In [13]:
# Create Boto3 client for IAM and STS

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
bedrock_agent_client = boto3.client('bedrock-agent')
boto_session = boto3.session.Session()

print(boto_session)

Session(region_name='us-east-1')


In [14]:
region = boto_session.region_name
account_id = sts_client.get_caller_identity()["Account"]
pprint.pprint(region)
pprint.pprint(account_id)

'us-east-1'
'288584671716'


In [15]:
# Agent foundation model name
BEDROCK_AGENRIC_FOUNDATION_MODEL = "anthropic.claude-3-5-sonnet-20240620-v1:0"

In [17]:
# Create bedrock agent IAM policy
agent_bedrock_allow_policy_name = "uc-ai-bra-iam-policy-name"

# Create IAM policies for the agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{BEDROCK_AGENRIC_FOUNDATION_MODEL}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

bedrock_agent_policy = iam_client.create_policy(
    PolicyName = agent_bedrock_allow_policy_name,
    PolicyDocument = bedrock_policy_json
)

In [18]:
# Create IAM Role for the agent and attach IAM policies
bedrock_agent_role_name = "uc-ai-bra-iam-role-name"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)

bedrock_agent_role = iam_client.create_role(
    RoleName = bedrock_agent_role_name,
    AssumeRolePolicyDocument = assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName = bedrock_agent_role_name,
    PolicyArn = bedrock_agent_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '784b5aa7-fe80-4880-ae5e-903e3279f485',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 24 Jan 2025 14:57:11 GMT',
   'x-amzn-requestid': '784b5aa7-fe80-4880-ae5e-903e3279f485',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [ ]:
pprint.pp(bedrock_agent_role)

In [20]:
# Create bedrock agent 

bedrock_agent_name = "uc-ai-bedrock-agent-name"
bedrock_agent_description = "uc-ai-bedrock-agent-description"
bedrock_agent_instruction = "You are a weather agent to fetch the current weather in celsius for a given location and date"

create_agent_response = bedrock_agent_client.create_agent(
    agentName = bedrock_agent_name,
    agentResourceRoleArn = bedrock_agent_role['Role']['Arn'],
    description = bedrock_agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel = BEDROCK_AGENRIC_FOUNDATION_MODEL,
    instruction = bedrock_agent_instruction,
)

In [ ]:
pprint.pprint(create_agent_response)

In [22]:
agent_id = create_agent_response['agent']['agentId']
print(agent_id)

HXPGL1POGD


In [23]:
# Bedrock Agent action group function specification 
bedrock_agent_functions = [

    {
        'name': 'get_weather_in_celsius',
        'description': 'Fetche the current weather in celsius for a given location',
        'parameters': {
            "location_id": {
                "description": "The unique identifier of the location to retrieve the temperature for",
                "required": True,
                "type": "integer"
            },
            "fetch_date": {
                "description": "The specific date for which the temperature needs to be retrieved",
                "required": True,
                "type": "string"
            }
        },
        'requireConfirmation':'ENABLED'
    },
]

In [ ]:
# Prepare agent group using function schema
bedrock_agent_action_group_name = "uc-ai-bedrock-action-group-name"
bedrock_agent_action_group_description = "Actions to fetch the weather of a given location for a given date"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=bedrock_agent_action_group_name,
    functionSchema={
        'functions': bedrock_agent_functions
    },
    description=bedrock_agent_action_group_description
)

pprint.pp(agent_action_group_response)

In [25]:
# Prepare agent 
prepare_agent_response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(prepare_agent_response)

{'ResponseMetadata': {'RequestId': 'a5cc66ae-f32f-4798-810e-ceb737901f12', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 24 Jan 2025 14:59:39 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'a5cc66ae-f32f-4798-810e-ceb737901f12', 'x-amz-apigw-id': 'E5eiaH8loAMEqNw=', 'x-amzn-trace-id': 'Root=1-6793aadb-4e71e9264870fd1656e9fbad'}, 'RetryAttempts': 0}, 'agentId': 'HXPGL1POGD', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 1, 24, 14, 59, 39, 910049, tzinfo=tzutc())}


In [ ]:
bedrock_agent_alias_name = "uc-ai-bedrock-agent-alias-name"
bedrock_agent_creation_response = bedrock_agent_client.create_agent_alias(
    agentAliasName=bedrock_agent_alias_name, 
    agentId=agent_id
    )

pprint.pp(bedrock_agent_creation_response)

In [27]:
agent_alias_id = bedrock_agent_creation_response['agentAlias']['agentAliasId']
print(agent_alias_id)

4TL4EPM5HT


In [ ]:
# Optional way to list and fetch the agent_alias_id 
bedrock_agent_alias_list = bedrock_agent_client.list_agent_aliases(agentId = agent_id)
agent_alias_id = bedrock_agent_alias_list['agentAliasSummaries'][0]['agentAliasId']

In [29]:
#agent_id = "AP5RQUVNTU"
#agent_alias_id = "O6EXN8DJVZ"
print(f"agent_id={agent_id}")
print(f"agent_alias_id={agent_alias_id}")

agent_id=HXPGL1POGD
agent_alias_id=4TL4EPM5HT


In [30]:
# Create a session with Bedrock agent
toolkit_session = uc_f_toolkit.create_session(agent_id=agent_id, agent_alias_id=agent_alias_id)
# Generate unique session ID
agent_session_id = str(uuid.uuid1())

In [31]:
# Invoke agent with a question
bedrock_invoke_agent_response_1 = toolkit_session.invoke_agent(
                input_text = "What is the weather for location 1234 and date of 2024-11-19",
                enable_trace = False,
                session_id = agent_session_id)

In [ ]:
pprint.pp(bedrock_invoke_agent_response_1)

In [ ]:
# Capute the event stream to process the response
event_stream = bedrock_invoke_agent_response_1['completion']

for event in event_stream:
   pprint.pp(event)

In [36]:
# Extract the return control function invocation inputs 

function_details = event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]


agent_function_name = function_details['function']


function_parameters = function_details['parameters']

In [37]:
pprint.pp(f"Function name given by the agent={agent_function_name}")
pprint.pp(f"Function parameters given by the agent={function_parameters}")

'Function name given by the agent=get_weather_in_celsius'
("Function parameters given by the agent=[{'name': 'fetch_date', 'type': "
 "'string', 'value': '2024-11-19'}, {'name': 'location_id', 'type': 'integer', "
 "'value': '1234'}]")


# Function Execution

## Listing Function

In [38]:
functions = uc_client.list_functions(
    catalog=CATALOG,
    schema=SCHEMA,
    max_results=10  # Paginated results will contain a continuation token that can be submitted with additional requests
)

pprint.pp(functions)

[FunctionInfo(name='bedrock_test_function', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='name', type_text='STRING', type_json='{"name": "name", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL', routine_definition='try:\n    # Fetch from Databricks SQL Warehouse based UC function execution \n    return "hello: " + name\nexcept Exception as e:\n    raise Exception(f"Error occurred: {e}")', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', i

In [39]:


for f in functions:
    if f.name == agent_function_name:
      print(f"Function: {f.name}")
      print(f"Parameters: {f.input_params}")

Function: get_weather_in_celsius
Parameters: parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "A unique identifier for the location."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='A unique identifier for the location.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date to fetch the weather for (format: YYYY-MM-DD)."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date to fetch the weather for (format: YYYY-MM-DD).')]


In [ ]:
# UC Response
# Parameters: 
# parameters=[
#     FunctionParameterInfo(
#         name='location_id', 
#         type_text='STRING', 
#         type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', 
#         type_name=<ColumnTypeName.STRING: 'STRING'>, 
#         type_precision=None, type_scale=None, 
#         type_interval_type=None, 
#         position=0, 
#         parameter_mode=None, 
#         parameter_type=None, 
#         parameter_default=None, 
#         comment='The name to be included in the greeting message.'), 
#     FunctionParameterInfo(
#         name='fetch_date', 
#         type_text='STRING', 
#         type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', 
#         type_name=<ColumnTypeName.STRING: 'STRING'>, 
#         type_precision=None, type_scale=None, 
#         type_interval_type=None, 
#         position=1, 
#         parameter_mode=None, 
#         parameter_type=None, 
#         parameter_default=None, 
#         comment='The date with the location')
#     ]


## Executing Function

In [47]:
parameters = {"location_id": "New Jersey", "fetch_date": "2025-01-01"}
result = uc_client.execute_function("location_weather_in_c", parameters)
pprint.pp(result)

ServiceException: (500)
Reason: Internal Server Error
HTTP response headers: <CIMultiDictProxy('Content-Type': 'application/json', 'Content-Length': '5622', 'Server': 'Armeria/1.28.4', 'Date': 'Fri, 24 Jan 2025 15:25:26 GMT')>
HTTP response body: {"error_code":"INTERNAL","details":[{"reason":"INTERNAL","metadata":{},"@type":"google.rpc.ErrorInfo"}],"stack_trace":"[java.base/java.util.Objects.requireNonNull(Objects.java:235), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:661), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:618), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:603), io.unitycatalog.server.service.FunctionService.getFunction(FunctionService.java:99), com.linecorp.armeria.internal.server.annotation.AnnotatedService.invoke(AnnotatedService.java:382), com.linecorp.armeria.internal.server.annotation.AnnotatedService.serve0(AnnotatedService.java:296), com.linecorp.armeria.internal.server.annotation.AnnotatedService.serve

In [42]:
# Prepare the parameters

exec_parameters = {item['name']: item['value'] for item in function_parameters}
pprint.pp(f"exec_parameters={exec_parameters}")

"exec_parameters={'fetch_date': '2024-11-19', 'location_id': '1234'}"


In [43]:
# Execution
exec_function_result = uc_client.execute_function(agent_function_name, exec_parameters)
pprint.pp(exec_function_result)


ServiceException: (500)
Reason: Internal Server Error
HTTP response headers: <CIMultiDictProxy('Content-Type': 'application/json', 'Content-Length': '5622', 'Server': 'Armeria/1.28.4', 'Date': 'Fri, 24 Jan 2025 15:07:51 GMT')>
HTTP response body: {"error_code":"INTERNAL","details":[{"reason":"INTERNAL","metadata":{},"@type":"google.rpc.ErrorInfo"}],"stack_trace":"[java.base/java.util.Objects.requireNonNull(Objects.java:235), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:661), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:618), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:603), io.unitycatalog.server.service.FunctionService.getFunction(FunctionService.java:99), com.linecorp.armeria.internal.server.annotation.AnnotatedService.invoke(AnnotatedService.java:382), com.linecorp.armeria.internal.server.annotation.AnnotatedService.serve0(AnnotatedService.java:296), com.linecorp.armeria.internal.server.annotation.AnnotatedService.serve

In [77]:

print(f"{agent_function_name} result = {exec_function_result.value}")

The weather in centigrade for New Jersey on 2025-01-01 is: 23


In [44]:
#Prepare agent session state 
#agent_invoke_response_body = f"function_name: {exec_function_result.value}"
agent_invoke_response_body = f"function_name: 23"
agent_session_state = {
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
            'functionResult': {
                'actionGroup': function_details["actionGroup"],
                'function': function_details["function"],
                'confirmationState': 'CONFIRM',
                'responseBody': {
                    "TEXT": {
                        'body': agent_invoke_response_body
                    }
                }
            }
        }]
    }

In [ ]:
time.sleep(70)
# Send result back to agent
bedrock_agent_final_response = toolkit_session.invoke_agent(
    input_text = "",
    session_id = agent_session_id,
    enable_trace = False,
    session_state = agent_session_state
)
pprint.pp(bedrock_agent_final_response)

In [46]:
# Print final response
print("Agent Response:")
for final_event in bedrock_agent_final_response.get('completion', []):
    print(f"{final_event}")

Agent Response:
{'chunk': {'bytes': b'The weather for location ID 1234 on 2024-11-19 is 23\xc2\xb0C (Celsius).'}}


In [ ]:
# Here's a breakdown:

# 23: Represents the temperature value.
# \xc2\xb0: The UTF-8 byte sequence for the degree symbol (°).
# C: Represents Celsius.
# When properly decoded, this string should display as 23°C.